In [7]:
import os
import sys
sys.path.append('./fortran')
import gibbsSampler6th
print gibbsSampler6th.gibbs_sampler.gibbssampler.__doc__
import numpy as np
import scipy as sp
from scipy.special import gammaln
import scipy.misc
import random
import numpy as np
import pandas as pd
import os
import glob
import random
random.seed(1234)
#
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.special import gammaln

def index_sample(p):
    """
    Desc: Samples from n topics distributed multinomially and returns topic number
    input: p - A one dimensional array of float64 type that contains the probability for each topic
    output: an Integer specifying which topic was chosen from a multinomial distribution
    """
    r = random.random()
    for i in range(len(p)):
        r = r - p[i]
        if r < 0:
            return i
    return len(p) - 1

def word_indices(vec):
    """
    Desc: Take a vector of word counts from a document and create a generator for word indices
    input: A vector from a Document Term Frequency matrix for one document.
    output: A generator object to store the word indices when called
    """
    for idx in vec.nonzero()[0]:
        for i in xrange(int(vec[idx])):
            yield idx

def log_multi_beta(alpha, K = None):
    """
    Desc: Compute the logarithm of the multinomial beta function
    input: alpha - A vector with type float64 or a scaler of float64
           K - An integer that, if alpha is a scalar, multiplies the log by K
    output: a float64 with value of the logarithm of the multinomial beta
    """

    if K is None:
        return np.sum(gammaln(alpha) - gammaln(np.sum(alpha)))
    else:
        return K * gammaln(alpha) - gammaln(K * alpha)

class LdaSampler(object):

    def __init__(self,  data, ntopics, alpha = .1, beta = .1):
        """
        Desc: Initialize values for our class object
        alpha: a float scalar
        beta: a float scalar
        ntopics: an integer for the number of topics
        """
        #if not isinstance(alpha, float):
        #    raise Exception(" Initial value for alpha must be a floating point number (.3)")

        #if not isinstance(beta, float):
        #    raise Exception(" Initial value for beta must be a floating point number (.3)")

        #if not isinstance(ntopics, int):
        #    raise Exception(" The number of topics must be an integer")

        self.matrix = data
        self.ntopics = ntopics
        self.alpha = alpha
        self.beta = beta
        self._initialize()
    def _initialize(self):
        """
        Initialize:
        NZM: size(#Docs X #Topics) numpy array with type float 64
            The number of times document M and topic Z interact

        NZW: size(#Topics X #Words) numpy array with type float64
            The number of times topic Z and word W interact

        NM:  size(#Docs) numpy array with type float64
            Sum of documents occurances by topic and word

        NZ:  size(#Topics) numpy array with type float64
            Sum of Topic occurences by word and document

        Topics: size(?) An empty set
           Will come back to this
        """
        ndocs, vsize = self.matrix.shape

        self.NZM = np.zeros((ndocs, self.ntopics))
        self.NZW = np.zeros((self.ntopics, vsize))
        self.NM  = np.zeros(ndocs)
        self.NZ  = np.zeros(self.ntopics)
        self.topics = []
        self.logL = []
        
        for m in xrange(ndocs):
            # Iterates over i, doc_length - 1, and w, the size of unique_words - 1
            for n in xrange(vsize):
                if self.matrix[m,n] == 0:
                    continue
                for w in xrange(self.matrix[m,n]):
                # Initialize a random topic for each word
                    z = np.random.randint(self.ntopics)
                    self.topics.append(z)
                    self.NZM[m,z] += 1
                # Why is NM being +1'd for each i,w?
                    self.NM[m] += 1
                    self.NZW[z,n] += 1
                    self.NZ[z] += 1
                # Keep document, iterator for word, word index, and assignment
                #self.topics.append([i,w,z])
        
        self.topics = np.vstack(self.topics)
    def _conditional_distribution(self, m, n):
        """
        Desc: Compute the conditional distribution of words in document and topic
        Input: m: An integer representing the column index of the document
               w: The generator object from word_indices

        Output: p_z: An array size(1 X ntopics) containing
                  probabilities for topics of word
        
        The formula is:
        ((n_{k,-i}^(t) + \beta_t)/\sum_{t=1}^V(n_{k,-i}^(t) + \beta_t)) *
        ((n_{m,-i}^(t) + \alpha_k)/(\sum_{k=1}^K(n_m^k + \alpha_k) - 1))
        """
        vsize = self.matrix[m,:].nonzero()[0].size
        p_z = np.zeros(self.ntopics)
        for ii in xrange(self.ntopics):
            p_z[ii] = (self.NZM[m,ii] + self.alpha) \
            *(self.NZW[ii,n] + self.beta) \
            / (self.NZ[ii] + vsize * self.beta)
        
        p_z /= np.sum(p_z)

        return p_z

    def loglikelihood(self):
        """
        Desc: Compute the log likelihood that the model generated the data
        Input: self references
        Output: lik: float of the log likelihood
        """
        # Why are these being repeated here?
        vsize = self.matrix[m,:].nonzero()[0].size
        ndocs = self.NZM.shape[0]
        lik = 0

        for z in xrange(self.ntopics):
            lik += log_multi_beta(self.NZW[z,:] + self.beta)
            lik -= log_multi_beta(self.beta, vsize)

        for m in xrange(ndocs):
            lik += log_multi_beta(self.NZM[m,:] + self.alpha)
            lik -= log_multi_beta(self.alpha, self.ntopics)

        return lik

    def phi_theta(self):
        """
        Desc: Compute phi and theta, our topic by word probs and document by topic probs
        Input: Self references
        Output: Two arrays, holding
            [0] phi: Probability of topic by word
            [1] theta: Probability of document by topic
        """
        

        num_phi = self.NZW + self.beta
        num_phi /= np.sum(num_phi, axis = 0)[np.newaxis,:]

        num_theta = self.NZM + self.alpha
        num_theta /= np.sum(num_theta,axis = 1)[:,np.newaxis ]

        return num_phi, num_theta


    def run(self, maxiter = 30, burnin= 0):
        """
        Desc: Perform Gibbs sampling for maxiter iterations

        Input: matrix - An array that is a Document Term Frequency Matrix
               maxiter - An integer with the number of iterations
               Burnin - TBA: An integer of the number of burnins

        Output: phi_theta() Two arrays, holding
        [0] Probability of topic by word
        [1] Probability of document by topic
        """

        n_docs, vsize = self.matrix.shape
        topics2 = self.topics


        for iteration in xrange(maxiter + 2):
            # Idea: After each iteration we now want to
            # make assignments relative to the newly generated topics
            if iteration > 1:
                self.topics = topics2
            for m in xrange(n_docs):
                for n in xrange(vsize):
                    if self.matrix[m,n] == 0:
                        continue
                    for w in xrange(self.matrix[m,n]):
                        
                        z = self.topics[m,n]
                    
                        self.NZM[m,z] -= 1
                        self.NM[m] -= 1
                        self.NZW[z,n] -= 1
                        self.NZ[z] -= 1

                        p_z = self._conditional_distribution(m,n)
                        # Choosing a random topic row
                        ind_z = np.random.randint(self.ntopics)
                    
                        # Sampling random topic
                        z = index_sample(p_z)
                        
                        #Self.topics needs to change after we iterate over this word
                        # Otherwise at each iteration we subtract one from that space w*n times
                        # giving us a negative number
                        topics2[m,n] = z

                        self.NZM[m,z] += 1
                        self.NM[m] += 1
                        self.NZW[z,n] += 1
                        self.NZ[z] += 1

            if iteration > burnin:
                yield self.phi_theta()

                
    def runfort(self, maxiter = 30, burnin= 0):
        """
        Desc: Perform Gibbs sampling for maxiter iterations

        Input: matrix - An array that is a Document Term Frequency Matrix
               maxiter - An integer with the number of iterations
               Burnin - TBA: An integer of the number of burnins

        Output: phi_theta() Two arrays, holding
        [0] Probability of topic by word
        [1] Probability of document by topic
        """

        M,N = self.matrix.shape

        p_z = np.zeros(self.ntopics)
        p_z += 1./self.ntopics
        
 
        # Make everything fortan contiguous
        p_z=p_z.flatten() # Flatten array (Make 1-D)
        p_z=p_z.reshape(self.ntopics, order='F')
        run_matrix = np.array(self.matrix.transpose(),order='F')
        run_NZM = np.array(self.NZM.transpose(),order='F')
        run_NZW = np.array(self.NZW,order='F')
        run_NZ = np.array(self.NZ,order='F')
        run_NM = np.array(self.NM,order='F')
        # index starts at 1 in fortran
        run_topics = np.array(self.topics,order='F') + 1
        run_topics2 = run_topics
        
        
        topics2 = self.topics.transpose()
        loglik = np.zeros(maxiter)
        
        gibbsSampler6th.gibbs_sampler.gibbssampler(matrix = run_matrix,
                                                nzw = run_NZW,
                                                nzm = run_NZM,
                                                nz = run_NZ,
                                                nm = run_NM,
                                                max_iter = maxiter,
                                                p_z = p_z,
                                                m = M,
                                                n = N,
                                                topics = run_topics,
                                                alpha = self.alpha,
                                                beta = self.beta,
                                                lik = loglik)
                                                

        #if iteration > burnin:
        return self.phi_theta(),loglik
                
                

    def prn(self,x = None):
        print x

    # For some reason this returns (maxiter - burnin) - 2 iterations?
    def update(self, maxiter = 20, burnin = 0):
        """
        Desc: Runs gibbs sampler for maxiter iterations
            Input: maxiter - integer specifying maximum number of iterations
                   burnin  - integer specifying number of iterations to burn through.
                                should be set to zero after initial burnin
            Output: phi_theta() Two arrays, holding
                [0] Probability of topic by word
                [1] Probability of document by topic
        """
        
        for iteration, phi_theta in enumerate(self.run( maxiter, burnin)):
            self.prn(iteration)
            self.prn(self.loglikelihood())
            self.logL.append(self.loglikelihood())
        return self.phi_theta(), self.logL

    def __call__(self):
        self.NZM = self.NZM
        self.NM = self.NM
        self.NZW = self.NZW
        self.NZ = self.NZ
        self.logL = self.logL

gibbssampler(matrix,nzw,nzm,nz,nm,p_z,topics,alpha,beta,lik,[ntopics,max_iter,m,n,top_size])

Wrapper for ``gibbssampler``.

Parameters
----------
matrix : input rank-2 array('q') with bounds (n,m)
nzw : input rank-2 array('q') with bounds (ntopics,n)
nzm : input rank-2 array('q') with bounds (ntopics,m)
nz : input rank-1 array('q') with bounds (ntopics)
nm : input rank-1 array('q') with bounds (m)
p_z : input rank-1 array('d') with bounds (ntopics)
topics : input rank-1 array('q') with bounds (top_size)
alpha : input float
beta : input float
lik : in/output rank-1 array('d') with bounds (max_iter)

Other Parameters
----------------
ntopics : input long, optional
    Default: shape(nzw,0)
max_iter : input long, optional
    Default: len(lik)
m : input long, optional
    Default: shape(matrix,1)
n : input long, optional
    Default: shape(matrix,0)
top_size : input long, optional
    Default: len(topics)



In [ ]:
# Grab made up data

import os
import shutil

N_TOPICS = 10
DOCUMENT_LENGTH = 100
FOLDER = "topicimg"

def vertical_topic(width, topic_index, document_length):
    """
    Generate a topic whose words form a vertical bar.
    """
    m = np.zeros((width, width))
    m[:, topic_index] = int(document_length / width)
    return m.flatten()

def horizontal_topic(width, topic_index, document_length):
    """
    Generate a topic whose words form a horizontal bar.
    """
    m = np.zeros((width, width))
    m[topic_index, :] = int(document_length / width)
    return m.flatten()

def save_document_image(filename, doc, zoom=2):
    """
    Save document as an image.
    doc must be a square matrix
    """
    height, width = doc.shape
    zoom = np.ones((width*zoom, width*zoom))
    # imsave scales pixels between 0 and 255 automatically
    sp.misc.imsave(filename, np.kron(doc, zoom))

def gen_word_distribution(ntopics, document_length):
    """
    Generate a word distribution for each of the ntopics.
    """
    width = ntopics / 2
    vsize = width ** 2
    m = np.zeros((ntopics, vsize))

    for k in range(width):
        m[k,:] = vertical_topic(width, k, document_length)

    for k in range(width):
        m[k+width,:] = horizontal_topic(width, k, document_length)

    m /= m.sum(axis=1)[:, np.newaxis] # turn counts into probabilities

    return m

def gen_document(word_dist, ntopics, vsize, length=DOCUMENT_LENGTH, alpha=0.1):
    """
    Generate a document:
    1) Sample topic proportions from the Dirichlet distribution.
    2) Sample a topic index from the Multinomial with the topic
       proportions from 1).
    3) Sample a word from the Multinomial corresponding to the topic
       index from 2).
    4) Go to 2) if need another word.
    """
    theta = np.random.mtrand.dirichlet([alpha] * ntopics)
    v = np.zeros(vsize)
    for n in range(length):
        z = index_sample(theta)
        w = index_sample(word_dist[z,:])
        v[w] += 1
    return v

def gen_documents(word_dist, ntopics, vsize, n=500):
    """
    Generate a document-term matrix.
    """
    m = np.zeros((n, vsize))
    for i in xrange(n):
        m[i, :] = gen_document(word_dist, ntopics, vsize)
    
    return m

if os.path.exists(FOLDER):
    shutil.rmtree(FOLDER)
os.mkdir(FOLDER)

width = N_TOPICS / 2
vocab_size = width ** 2
word_dist = gen_word_distribution(N_TOPICS, DOCUMENT_LENGTH)
matrix = gen_documents(word_dist, N_TOPICS, vocab_size)


In [ ]:
sampler = LdaSampler(data = matrix.astype(int), ntopics = 3, alpha =.01,
                     beta = .01)

In [ ]:
LDA_fort_test = sampler.runfort(maxiter=1000)

In [ ]:
LDA_fort_test

In [ ]:
testing_frame = np.array([[1,0,4,2],[2,4,0,5]])
sampler = LdaSampler(data = testing_frame, ntopics = 3, alpha =.1,
                     beta = .1)
p_z = np.zeros(3) + .33
M,N = testing_frame.shape

In [ ]:
LDA_fort_test = sampler.runfort(maxiter=1)

In [ ]:
LDA_fort_test

In [ ]:
print "matrix"
print sampler.matrix.transpose().shape
print "topics"
print sampler.topics.transpose().shape
print "NZ"
print sampler.NZ.transpose().shape
print "NM"
print sampler.NM.transpose().shape
print "NZM"
print sampler.NZM.transpose().shape
print "NZW"
print sampler.NZW.transpose().shape

In [ ]:
p_z=p_z.flatten() # Flatten array (Make 1-D)
p_z=p_z.reshape(sampler.ntopics, order='F')

In [ ]:
p_z

In [ ]:
topics2 = sampler.topics.transpose()
alpha = .1
beta = .1
gibbsSampler.gibbs_sampler.gibbssampler(matrix = sampler.matrix.transpose(),
                                                nzw = sampler.NZW.transpose(),
                                                nzm = sampler.NZM.transpose(),
                                                nz = sampler.NZ,
                                                nm = sampler.NM,
                                                max_iter = 10,
                                                p_z = p_z,
                                                m = M,
                                                n = N,
                                                topics = sampler.topics.transpose(),
                                                topics2 = topics2,
                                                alpha = alpha,
                                                beta = beta)

In [ ]:
arr = np.array([[1,2,3,4],[1,2,3,4]])
print arr
arr2 = np.delete(arr, 1,axis=1)
print arr2

In [8]:

#
DIR = r'data_folder/wordcounts'
allfiles = glob.glob(os.path.join(DIR,"*.CSV"))
p=.8
# sample files for train
gen_sample = np.array(sorted(random.sample(xrange(len(allfiles)), int(p * len(allfiles)))))
rand_sample = [ allfiles[i] for i in gen_sample ]
#
# take rest for test
rand_sample2 = []
for i in xrange(len(allfiles)):
    if i not in gen_sample:
        rand_sample2.append(allfiles[i])
#
# train data

np_array_list = []
for file_ in rand_sample:
    df = pd.read_csv(file_,index_col=None, header=0)
    df['source'] = file_
    np_array_list.append(df.as_matrix())
#
# test data
np_array_list_test = []
for file_ in rand_sample2:
    df = pd.read_csv(file_, index_col = None, header = 0)
    df['source'] = file_
    np_array_list_test.append(df.as_matrix())
    
#
# train data frame
comb_np_array = np.vstack(np_array_list)
train_frame = pd.DataFrame(comb_np_array)
train_frame.columns = ['words','count','source']
subless = (train_frame['words'].str.len() > 2)
submore = (train_frame['words'].str.len() < 20)
train_frame = train_frame.loc[subless]
train_frame = train_frame.loc[submore]
train_frame = train_frame.fillna(value = 0)
train_frame = train_frame.pivot(index = 'source',columns = 'words', values = 'count')
train_frame = train_frame.fillna(value = 0)
train_frame = train_frame.loc[:, (train_frame.sum(axis = 0) > 10)]
#

# test data frame
comb_np_array_test = np.vstack(np_array_list_test)
test_frame = pd.DataFrame(comb_np_array_test)
test_frame.columns = ['words','count','source']
test_frame = test_frame.fillna(value=0)
test_frame = test_frame.pivot(index = 'source', columns = 'words', values = 'count')
test_frame = test_frame.fillna(value = 0)

In [9]:
train_frame.shape

(800, 8569)

In [10]:
test_frame1 = train_frame.values.astype(int)

In [11]:
sampler1 = LdaSampler(data = test_frame1, ntopics = 5, alpha = .0001, beta = .0001)


In [ ]:
test_lda = sampler1.runfort( maxiter =30000, burnin = 0)

In [6]:
test_lda

((array([[  1.00002500e-01,   2.54901423e-01,   3.02324392e-01, ...,
            8.33381942e-02,   1.17649481e-01,   9.09140494e-02],
         [  1.00002500e-01,   5.88249135e-02,   1.39535587e-01, ...,
            3.33327778e-01,   2.35293080e-01,   3.63628926e-01],
         [  3.49996250e-01,   2.94116724e-01,   1.16280043e-01, ...,
            1.66668055e-01,   4.11758478e-01,   3.63628926e-01],
         [  3.49996250e-01,   2.15686121e-01,   2.32557761e-01, ...,
            4.16657639e-01,   1.17649481e-01,   9.09049589e-06],
         [  1.00002500e-01,   1.76470819e-01,   2.09302217e-01, ...,
            8.33298613e-06,   1.17649481e-01,   1.81819008e-01]]),
  array([[ 0.14893644,  0.30851006,  0.1382982 ,  0.25531885,  0.14893644],
         [ 0.1746032 ,  0.19246032,  0.17857145,  0.2142857 ,  0.24007933],
         [ 0.21153845,  0.19642857,  0.18131869,  0.22527471,  0.18543957],
         [ 0.19115981,  0.19720438,  0.20249339,  0.20249339,  0.20664904],
         [ 0.20210186,  

In [ ]:
print "matrix"
print sampler1.matrix.shape
print "topics"
print sampler1.topics.shape
print "NZ"
print sampler1.NZ.shape
print "NM"
print sampler1.NM.shape
print "NZM"
print sampler1.NZM.shape
print "NZW"
print sampler1.NZW.shape

In [ ]:
sampler.matrix

In [ ]:
sampler.update( maxiter =2000, burnin = 0)

In [ ]:
m=1
n=2
vsize = sampler.NZW[m,:].nonzero()[0].size
p_z = np.zeros(sampler.ntopics)
for ii in xrange(sampler.ntopics):
    p_z[ii] = (sampler.NZM[m,ii] + sampler.alpha) \
    *(sampler.NZW[ii,n] + sampler.beta) \
    / (sampler.NZ[ii] + vsize * sampler.beta)
        
#p_z /= np.sum(p_z)

In [ ]:
p_z /= np.sum(p_z)

In [ ]:
p_z

In [ ]:
testtt = np.array([[ 1.125,       1.125 ,      0.32142857 , 0.32142857],
 [ 1.30357143 , 1.30357143 , 0.37244898  ,0.37244898],
 [ 8.44642857,  8.44642857,  2.41326531 , 2.41326531],
 [ 5.03125,     5.03125 ,    1.4375 ,     1.4375    ]])

In [ ]:
sampler.update( maxiter =1, burnin = 0)

In [ ]:
print "matrix"
print sampler.matrix.shape
print "topics"
print sampler.topics.shape
print "NZ"
print sampler.NZ.shape
print "NM"
print sampler.NM.shape
print "NZM"
print sampler.NZM.shape
print "NZW"
print sampler.NZW.shape


In [ ]:
gibbsSampler.gibbs_sampler.gibbssampler(matrix = sampler.matrix, nzw = sampler.NZW.transpose(),
                                                nzm = sampler.NZM.transpose(),
                                                nz = sampler.NZ,
                                                nm = sampler.NM,
                                                max_iter = 10,
                                                p_z = p_z,
                                                m = M,
                                                n = N,
                                                topics = sampler.topics,
                                                topics2 = topics2)

In [ ]:
for m in xrange(testing_frame.shape[0]):
    for i,w in enumerate(word_indices(testing_frame[m,:])):
        print i,w

In [ ]:
sampler = LdaSampler(data = train_frame.values, ntopics = 4, alpha = .1, beta = .1)
LDAtest = sampler.runfort(maxiter = 8)

In [ ]:
gibbsSampler.gibbs_sampler.gibbssampler(matrix = sampler.matrix, nzw = sampler.NZW.transpose(),
                                                nzm = sampler.NZM.transpose(),
                                                nz = sampler.NZ,
                                                nm = sampler.NM,
                                                max_iter = 10,
                                                p_z = p_z,
                                                m = M,
                                                n = N)

In [ ]:
sampler.NZM